In [ ]:
import numpy as np
from numpy import random
from numpy import linalg
import math
import matplotlib.pyplot as plt

In [ ]:
# Defining a robust method to construct a 2D discrete Laplacian with periodic boundary conditions

def lap():
    
    Msqr = int(np.sqrt(M));

    D = np.eye(Msqr,Msqr,1) + np.eye(Msqr,Msqr,-1) - 2*np.identity(Msqr);
    D[0][Msqr-1] += 1;
    D[Msqr-1][0] += 1;
    K = np.kron(D,np.identity(Msqr))+np.kron(np.identity(Msqr),D);
    K = K + np.identity(M);
    return K;

In [ ]:
# Storing stationary points for M = 4, 9

def stationary():
    
    dicts = {4: [-0.07090414827752234]*4, 9: [0.261322]*9};
    
    return dicts[M];

In [ ]:
# Evaluating the action at a point x

def action(x):
    
    xnew = x/np.sqrt(N);
    phix = staarray + xnew;
    quad = np.sum(np.square(phix))/2;
    
    sign = (-1)**(M+1);
    
    det = np.linalg.det(np.diag(phix) + K);
    
    # Calculating the constant and linear terms so that they can be subtracted
    
    constant = np.log(sign*np.linalg.det(np.diag(staarray) + K)) + np.sum(np.square(staarray))/2;
    
    actionlinear = np.linalg.inv(np.diag(staarray) + K);
    actionlinearcontribution = 0;
    
    for i in range(M):
        
        actionlinearcontribution += (actionlinear.item((i,i))+staarray[i]) * np.sqrt(N);
    
    return N*(quad + np.log(sign*det) - constant - actionlinearcontribution);

In [ ]:
# Checking if a value site is updated based on the change in action dS

def aux(dS):
    
    if dS < 0:
        
        return True;
    
    prob = random.uniform(0,1);
    R = min(1,math.e**(-dS));
    
    return True if prob <= R else False; 

In [ ]:
# Method to thermalize

def therm(start):
    
    # 2000 runs per thermalization
    Ntherm = 2000;
    Nacc = 0;
    x = np.zeros((1,M));
    x[0] = start;
    
    xaxis = [];
    yaxis = [];
    
    xaxis = [0];
    yaxis = [start[2]];
    
    # Here, 2*Ar is the size of the interval from which we choose x2 from x
    Ar = 2.75;
    
    # Counter to keep track of probability of acceptance
    counter = 0;

    for i in range(1, Ntherm + 1):
        
        newrow = x[i - 1];
        x = np.vstack([x,newrow]);

        for j in range(len(x[0])):
            
            counter += 1;

            delta = random.uniform(-Ar,Ar);

            x2 = np.copy(x[i]);
            x2[j] += delta;

            dS = action(x2) - action(x[i]);

            if (aux(dS)):
                
                x[i] = x2;
                Nacc += 1;
        
        if i % 250 == 0:
            
            xaxis.append(i);
            yaxis.append(np.mean(x[-250:,2]));
            
    print("Probability of Acceptance: " + str(Nacc/counter))

    return x[len(x) - 1], xaxis, yaxis;

In [ ]:
def run():

    x1, xaxis, yaxis1  = therm(np.array([0]*M));
    x2, xaxis, yaxis2  = therm(np.array([-2]*M));
    x3, xaxis, yaxis3  = therm(np.array([4]*M));
    
    # Here, we chose to plot <x(3)> WLOG
    plt.plot(xaxis, yaxis1, '-b', label = '0');
    plt.plot(xaxis, yaxis2, '--r', label = '-20');
    plt.plot(xaxis, yaxis3, '-.k', label = '-3');
    plt.legend();
    
    plt.xlabel("Number of Updates")
    plt.ylabel('<x(3)>')
    
    plt.savefig("thermalizationQUADN" + str(N) + "M" + str(M) + ".png", dpi=600);
    plt.show();
    
    # We start the measurements phase with the average x of all three thermalizations
    x = (x1 + x2 + x3)/3;
    
    return measurements(x);

In [ ]:
def measurements(xtherm):
    
    # We follow a similar process for measurements with 200,000 total measurements
    xstore = np.zeros((1,M));
    xstore[0] = xtherm;
    
    Nmc = 2*10**5;
    
    length = int(M*(M+1)/2);
    
    xixj = np.zeros((1,length));
    
    # acounter to keep track observables order
    acounter = 0;
    
    for k in range(M):
        
        for l in range(k + 1):
            
            xixj[0][acounter] = xstore[0][k] * xstore[0][l];
            acounter += 1;
    
    xaxis = [];
    yaxis = [];
    yaxissqr = [];
    error = [];
    errorsqr = [];

    Ar = 2.75;
    
    # Counter to mark when to calculate autocorrelations
    spacingcounter = True;
    
    # Random element to plot
    length = int(M*(M+1)/2);
    randomelement = random.randint(0,length - 1);

    for i in range(1, Nmc + 1):
        
        newrow = xstore[i - 1];
        xstore = np.vstack([xstore,newrow]);
        
        newrow2 = xixj[i - 1];
        xixj = np.vstack([xixj,newrow2]);

        for j in range(len(xstore[0])):

            delta = random.uniform(-Ar,Ar);

            x2 = np.copy(xstore[i]);
            x2[j] += delta;

            dS = action(x2) - action(xstore[i]);

            if (aux(dS)):

                xstore[i] = x2;
        
        acounter = 0;
        
        for k in range(M):
            
            for l in range(k + 1):
                
                xixj[i][acounter] = xstore[i][k] * xstore[i][l];
            
                acounter += 1;
          
        if i % 10000 == 0:

            if spacingcounter:

                spacingarray = autocor(xixj);
                print(f"Spacing Array: {spacingarray}")
                spacing = max(spacingarray);
                print("Max Spacing: " + str(spacing));

                spacingcounter = False;

            xaxis.append(i);
            yaxis.append(np.mean(xixj[::spacingarray[randomelement],randomelement]));
            error.append(np.sqrt(spacingarray[9]*np.var(xixj[::spacingarray[randomelement],randomelement])/(i)));
            
            # Checkpoint, since there are a lot of observables
            print("Checkpoint: " + str(100*i/Nmc) + "% done.")
    
    # Plot of random element 
    plt.errorbar(xaxis, yaxis, yerr = error, fmt = 'o', capsize = 5, ecolor = 'red');
    plt.xlabel("Number of Measurements");
    plt.ylabel(arraycounter[randomelement]);
    plt.savefig("MCMCQUADN" + str(N) + "M" + str(M) + ".png", dpi=600);
    plt.show();

    return [np.mean(xixj[::spacingarray[k],k]) for k in range(length)], [np.sqrt(spacingarray[k]*np.var(xixj[::spacingarray[k],k])/(i)) for k in range(length)];

In [ ]:
# Calculate \rho_h

def R(col, h):
    
    sum1 = 0;
    
    for i in range(len(col) - h):
        
        sum1 += (col[i] - np.mean(col))*(col[i + h] - np.mean(col));
        
    sum1 = sum1 / (len(col) - h);
    
    return sum1;

In [ ]:
# Calculates \rho_h until this value drops below 0.1

def autocor(x):
    
    jarray = [];
    
    length = int(M*(M+1)/2);
    
    # Random element to plot the autocorrelation of
    graph = random.randint(0,length - 1);
    print("Autocorrelation of " + arraycounter[graph]);
    xaxisautocor = [0];
    yaxisautocor = [];
    
    for i in range(len(x[0])):
        
        col = x[:,i];

        samplemean = np.mean(col);

        norm = R(col, 0);
        
        if i == graph:
            
            yaxisautocor.append(1);

        for j in range(len(col)):

            yval = R(col, j)/norm;
            
            if i == graph:
                
                xaxisautocor.append(j);
                yaxisautocor.append(yval);

            if yval < 0.1:
                
                jarray.append(j);
                break;
    
    plt.plot(xaxisautocor, yaxisautocor, '-ob');
    plt.xlabel("h");
    plt.ylabel(r"$\rho$" + "(h)");
    
    plt.savefig("autocorQUADN" + str(N) + "M" + str(M) + ".png", dpi=600);
    plt.show();
    
    return jarray;

In [ ]:
# List of N that we wish to set. The M below can be changed for the respective lattice size
Nlist = [100, 10**3, 10**6];
M = 9;

# Define the Laplacian and stationary point depending on M
K = lap();
staarray = stationary();

# Keeping track of observables' order for printing
arraycounter = [];

for k in range(M):
    
        for l in range(k + 1):
            
            arraycounter.append("<x(" + str(l + 1) + ")x(" + str(k + 1) + ")>");

# Find values for the observables for each value of N. We also print it out neatly
for n in Nlist:
    
    N = n;
    
    string = "N = " + str(N);
    print(string.center(50,"="));
            
    mean, error = run();
    
    print(mean);

    for k in range(len(mean)):

        print(arraycounter[k] + "  =  " + "{:.3e}".format(mean[k]) + "  " + u"\u00B1" + "  " + "{:.3e}".format(error[k]));